# RLAgent: Automating tasks on Neuroglancer

This code provides the infrastructure to use Neuroglancer as a fully automated agent. This heavily relies on the Selenium library and JavaScript to control Neuroglancer's state. This provides a flexible and fast way to interact with Neuroglancer, with possibilities to do headless, local or remote sessions. More importantly, it allows to simulate a human's interaction and have a real environment for Reinforcement Learning training.

## The main kind of interaction is to:
- **click** (left, right, middle, double) at a specific position
- change the **JSON state** programmatically to navigate on continous dimensions (zoom, rotation, translation), add new layers, etc.

## It can:
- follow a recorded of sequence of clicks and JSON state changes done by a human
- acquire screenshots in less than 5 ms
- prepares the infrastructure to use Reinforcement Learning to navigate the Neuroglancer UI. However, this task is technically challenging. And has been decomposed into smaller tasks.

# Prerequisites

Before running this notebook, ensure you have the following dependencies installed:

- **Python**:(Version used: **3.10.16**)
- **Selenium**: (Version used: **4.29.0**)
- **Pillow**: (Version used: **11.1.0**)
- **numpy**: (Version used: **2.2.4**)
- **scipy**: (Version used: **1.12.1**)
- **torch**

We have provided already two chrome drivers for MacOS and Linux. It is very probably that you will need to install the correct version for your system. For this, you need to access Chrome and find your current version and then download the correct driver [here](https://googlechromelabs.github.io/chrome-for-testing/).

# Neuroglancer Automation:

The main class is environment.py which includes all necessary libraries to interact with ChromeNGL and Selenium

In [2]:
from environment import Environment

ImportError: attempted relative import with no known parent package

In [ ]:
#Starting a new Session
print("Starting session...")
env = Environment(headless=False)
env.start_session()

In [ ]:
# Get the current state
(pos_state, curr_image), json_state = env.prepare_state()
print(f"Position state: {pos_state}")
print(f"Image size: {curr_image.size}")

# Interacting Programmatically with `Agent.apply_actions()`

The `Agent.apply_actions()` function is designed to handle interactions by applying incremental changes to a JSON structure and specifying raw positions for mouse clicks.

### Action Vector Structure

The function takes a vector with the following components:

```plaintext
(
    left_click, right_click, double_click,  # 3 booleans for mouse clicks
    x, y,                                  # 2 floats for absolute mouse position
    key_Shift, key_Ctrl, key_Alt,          # 3 booleans for modifier keys
    json_change,                           # 1 boolean indicating a JSON change
    delta_position_x, delta_position_y, delta_position_z,  # 3 floats for position deltas
    delta_crossSectionScale,               # 1 float for cross-section scaling
    delta_projectionOrientation_q1, delta_projectionOrientation_q2,
    delta_projectionOrientation_q3, delta_projectionOrientation_q4,  # 4 floats for orientation quaternion
    delta_projectionScale                  # 1 float for projection scaling
)


In [ ]:
# Define a sample action vector
action_vector = [
    0, 0, 0,  # left, right, double click (all False)
    100, 100,  # x, y mouse position
    0, 0, 0,  # key_Shift, key_Ctrl, key_Alt (all False)
    1,  # json_change = True, meaning we want to modify the JSON state
    10, 0, 0,  # delta_position_x, delta_position_y, delta_position_z
    0,  # delta_crossSectionScale
    0.1, 0, 0, 0,  # delta_projectionOrientation_q1, q2, q3, q4
    1000  # delta_projectionScale
]
#Applying the action
print("Applying action...")
env.apply_actions(action_vector)

# Wait a moment to see the effect
time.sleep(1)

# Clean up
env.end_session()
print("Environment session ended")

For an environment test, we can define a loop. And use a model to take the action decision.

In [ ]:
# Example loop
for i in range(100):
    # Get the current state
    (pos_state, curr_image), json_state = env.prepare_state(
        image_path=None, 
        euler_angles=True, 
        resize=False, 
        add_mouse=False, 
        fast=True
    )
    
    # For example purposes, using a predefined action vector
    action_vector = [
        0, 0, 0,  # left, right, double click booleans
        100, 100,  # x, y mouse position
        0, 0, 0,  # no modifier keys
        1,  # json_change = True
        10, 0, 0,  # position change
        0,  # cross-section scaling
        0.2, 0, 0,  # orientation change in Euler angles
        2000  # projection scaling (log-scale in neuroglancer)
    ]
    
    # Apply the action
    env.apply_actions(action_vector, json_state=json_state, euler_angles=True)


In [ ]:
#We can also follow an episode that has been recorded by a human
episode_path = "./episodes/sample_episode.json"
env.follow_episode(episode_path, sleep_time=0.1)

#If we need to parse an episode from its recorded sequence of JSON states and actions, we can use the parse_episode function.
episode_path = "./episodes/raw/1800x900/episode_5.json"
save_path = "./reparsed_episodes/test/"
parsed_data = env.parse_episode(episode_path, save_path)

# Reinforcement Learning Framework: 

In [ ]:
# Initialize environment
env = Environment(headless=False, verbose=True)
env.start_session()

# Define action vector
action_vector = [
    0, 0, 0,  # left, right, double click
    100, 100,  # x, y
    0, 0, 0,  # no modifier keys
    1,  # json_change = True
    10, 0, 0,  # position change
    0,  # cross-section scaling
    0.1, 0, 0, 0,  # orientation change
    1000  # projection scaling
]

# Take a step
state, reward, done, json_state = env.step(action_vector)